In [1]:
import os
import xml.etree.ElementTree as ET
import shutil
import random
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from IPython.display import display, Image as IPImage
from ultralytics import YOLO


In [2]:
project_root = os.getcwd()

input_path = os.path.join(project_root, "grocery-shelves")
xml_file = os.path.join(input_path, "annotations.xml")
images_path = os.path.join(input_path, "img")

output_path = os.path.join(project_root, "yolo_output")
output_labels_dir = os.path.join(output_path, "yolo_labels")
dataset_path = os.path.join(output_path, "dataset")

os.makedirs(output_labels_dir, exist_ok=True)

In [3]:
tree = ET.parse(xml_file)
root = tree.getroot()

for image in root.findall('image'):
    img_name = image.get('name').split('/')[-1].split('.')[0]
    img_width = float(image.get('width'))
    img_height = float(image.get('height'))

    yolo_lines = []
    for polyline in image.findall('polyline'):
        points = polyline.get('points').split(';')
        x_coords = [float(p.split(',')[0]) for p in points]
        y_coords = [float(p.split(',')[1]) for p in points]

        x_min, x_max = max(0, min(x_coords)), min(img_width, max(x_coords))
        y_min, y_max = max(0, min(y_coords)), min(img_height, max(y_coords))

        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height
        x_center = ((x_min + x_max) / 2) / img_width
        y_center = ((y_min + y_max) / 2) / img_height

        if width <= 0 or height <= 0:
            continue

        yolo_line = f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
        yolo_lines.append(yolo_line)

    with open(os.path.join(output_labels_dir, f"{img_name}.txt"), 'w') as f:
        f.write('\n'.join(yolo_lines))

print(" Conversion to YOLO format completed.")


 Conversion to YOLO format completed.


In [4]:
for split in ['train', 'val']:
    os.makedirs(os.path.join(dataset_path, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(dataset_path, 'labels', split), exist_ok=True)

all_images = [f for f in os.listdir(images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
random.shuffle(all_images)
split_index = int(len(all_images) * 0.8)
train_images = all_images[:split_index]
val_images = all_images[split_index:]

def move_files(image_list, split):
    for img_file in image_list:
        label_file = img_file.rsplit('.', 1)[0] + '.txt'
        shutil.copy2(os.path.join(images_path, img_file), os.path.join(dataset_path, 'images', split))
        if os.path.exists(os.path.join(output_labels_dir, label_file)):
            shutil.copy2(os.path.join(output_labels_dir, label_file), os.path.join(dataset_path, 'labels', split))

move_files(train_images, 'train')
move_files(val_images, 'val')

print(" Dataset organized into train/val folders.")


 Dataset organized into train/val folders.


In [5]:
dataset_yaml = f"""\
path: {dataset_path}
train: images/train
val: images/val
names:
  0: box
"""

yaml_path = os.path.join(output_path, 'dataset.yaml')
with open(yaml_path, 'w') as f:
    f.write(dataset_yaml)

print(" YOLOv8 configuration file created at:", yaml_path)


 YOLOv8 configuration file created at: /Users/trishab/Desktop/Projects/yolov8-grocery-project/yolo_output/dataset.yaml


In [6]:
os.environ["WANDB_MODE"] = "disabled"
model = YOLO("yolov8s.pt") 

results = model.train(
    data=yaml_path,
    epochs=60,
    batch=16,
    imgsz=640,
    workers=2,
    project='grocery-shelves',
    name='yolov8s-results',
    exist_ok=True,
    verbose=True
)

print(" Training completed.")


Ultralytics 8.3.105 🚀 Python-3.10.10 torch-2.6.0 CPU (Apple M2 Pro)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/Users/trishab/Desktop/Projects/yolov8-grocery-project/yolo_output/dataset.yaml, epochs=60, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=grocery-shelves, name=yolov8s-results, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show

train: Scanning /Users/trishab/Desktop/Projects/yolov8-grocery-project/yolo_output/dataset/labels/train.cache... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]
val: Scanning /Users/trishab/Desktop/Projects/yolov8-grocery-project/yolo_output/dataset/labels/val... 11 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11/11 [00:00<00:00, 421.02it/s]

val: New cache created: /Users/trishab/Desktop/Projects/yolov8-grocery-project/yolo_output/dataset/labels/val.cache


Plotting labels to grocery-shelves/yolov8s-results/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to grocery-shelves/yolov8s-results
Starting training for 60 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/60         0G      1.354      2.438      1.418        194        640: 100%|██████████| 1/1 [00:06<00:00,  6.92s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.53s/it]

...


      60/60         0G     0.2997      0.348     0.7639         90        640: 100%|██████████| 1/1 [00:05<00:00,  5.91s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.40s/it]

                   all         11         76      0.999          1      0.995      0.972

60 epochs completed in 0.130 hours.
Optimizer stripped from grocery-shelves/yolov8s-results/weights/last.pt, 22.5MB


Optimizer stripped from grocery-shelves/yolov8s-results/weights/best.pt, 22.5MB

Validating grocery-shelves/yolov8s-results/weights/best.pt...
Ultralytics 8.3.105 🚀 Python-3.10.10 torch-2.6.0 CPU (Apple M2 Pro)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.32s/it]


                   all         11         76      0.999          1      0.995      0.972
Speed: 0.5ms preprocess, 96.5ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to grocery-shelves/yolov8s-results
 Training completed.


In [7]:
def plot_training_sample(img_path, label_path):
    img = cv2.imread(img_path)
    h, w = img.shape[:2]
    
    with open(label_path, 'r') as f:
        lines = f.readlines()
        
    for line in lines:
        cls, x_center, y_center, width, height = map(float, line.strip().split())
        x1 = int((x_center - width / 2) * w)
        y1 = int((y_center - height / 2) * h)
        x2 = int((x_center + width / 2) * w)
        y2 = int((y_center + height / 2) * h)
        
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Sample Training Image with Bounding Boxes")
    plt.axis('off')
    plt.show()

sample_train_img = os.path.join(dataset_path, 'images', 'train', train_images[0])
sample_train_label = os.path.join(dataset_path, 'labels', 'train', train_images[0].rsplit('.', 1)[0] + '.txt')
plot_training_sample(sample_train_img, sample_train_label)


<Figure size 800x600 with 1 Axes>

In [9]:
print(" Running inference on validation sample:")
sample_val_img = os.path.join(dataset_path, 'images', 'val', val_images[0])

results = model.predict(source=sample_val_img, conf=0.25, save=True)
result = results[0]

img = cv2.imread(sample_val_img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

for box in result.boxes.xyxy:
    x1, y1, x2, y2 = map(int, box.tolist())
    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)

plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.title(f"Predicted Objects: {len(result.boxes)}")
plt.axis('off')
plt.show()

print(f"Inference done. Total predicted objects: {len(result.boxes)}")


 Running inference on validation sample:

image 1/1 /Users/trishab/Desktop/Projects/yolov8-grocery-project/yolo_output/dataset/images/val/16.jpg: 480x640 3 boxs, 89.4ms
Speed: 2.2ms preprocess, 89.4ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
Results saved to grocery-shelves/yolov8s-results


<Figure size 800x600 with 1 Axes>

Inference done. Total predicted objects: 3


In [10]:
metrics = model.val(data=yaml_path)

print("\n YOLOv8 Evaluation Metrics:")
print(f"Precision:       {metrics.box.mp():.4f}" if callable(metrics.box.mp) else f"Precision:       {metrics.box.mp:.4f}")
print(f"Recall:          {metrics.box.mr():.4f}" if callable(metrics.box.mr) else f"Recall:          {metrics.box.mr:.4f}")
print(f"mAP@0.5:         {metrics.box.map50():.4f}" if callable(metrics.box.map50) else f"mAP@0.5:         {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95:    {metrics.box.map():.4f}" if callable(metrics.box.map) else f"mAP@0.5:0.95:    {metrics.box.map:.4f}")


Ultralytics 8.3.105 🚀 Python-3.10.10 torch-2.6.0 CPU (Apple M2 Pro)


val: Scanning /Users/trishab/Desktop/Projects/yolov8-grocery-project/yolo_output/dataset/labels/val.cache... 11 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11/11 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.26s/it]


                   all         11         76      0.999          1      0.995      0.972
Speed: 0.5ms preprocess, 89.3ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to grocery-shelves/yolov8s-results

 YOLOv8 Evaluation Metrics:
Precision:       0.9988
Recall:          1.0000
mAP@0.5:         0.9950
mAP@0.5:0.95:    0.9719
